<a href="https://colab.research.google.com/github/ArpitKadam/ColabNotebooks/blob/main/Text_Generation_with_GPT2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2-large")
model = AutoModelForCausalLM.from_pretrained("openai-community/gpt2-large").to(device)

In [ ]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1280)
    (wpe): Embedding(1024, 1280)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-35): 36 x GPT2Block(
        (ln_1): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=3840, nx=1280)
          (c_proj): Conv1D(nf=1280, nx=1280)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=5120, nx=1280)
          (c_proj): Conv1D(nf=1280, nx=5120)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1280, out_features=50257, bias=False)
)

# **BEAM SEARCH**

In [ ]:
input_text = "I am really impressed by your work and"
max_length = 256

input_ids = tokenizer(input_text, return_tensors="pt").to(device)

In [ ]:
input_ids

{'input_ids': tensor([[   40,   716,  1107, 12617,   416,   534,   670,   290]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}

In [ ]:
genrated_tensors = model.generate(
    input_ids['input_ids'],
    max_length=max_length,
    num_beams=5,
    do_sample=False,
    no_repeat_ngram_size=3
  )

print(genrated_tensors)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


tensor([[   40,   716,  1107, 12617,   416,   534,   670,   290,   314,  2911,
           284,   766,   517,   286,   340,   287,   262,  2003,    13,   198,
           198, 10449,   345,   845,   881,   329,   477,   534,  1327,   670,
           290, 22445,     0,   198,   198, 13014, 13957,    11,   198,   198,
            50,  1765,   459,   666,   198,   198, 17250, 26190,    11,   628,
           198,  9690,   329,  2263,   262,   640,   284,  3280,   616,  2683,
            13,   628,   198,  5962,   286,   477,    11,   314,   561,   588,
           284,  5875,   345,   329,   477,   262,   670,   345,   423,  1760,
           287,   262,  1613,  1178,   812,    13,   314,   423,   587,   257,
          4336,   286,   534,   670,   329,   257,   890,   640,    11,   290,
           314,   716,   845,  3772,   284,   766,   326,   345,   389,   991,
          1762,   319,   340,    13,   314,   561,   635,   588,   284,   910,
           326,   314,   716,   257,  3236,  4336,  

In [ ]:
text = tokenizer.decode(genrated_tensors[0], skip_special_tokens=True)
text

'I am really impressed by your work and I hope to see more of it in the future.\n\nThank you very much for all your hard work and dedication!\n\nBest regards,\n\nSebastian\n\nHi Sebastian,\n\n\nThanks for taking the time to answer my questions.\n\n\nFirst of all, I would like to thank you for all the work you have done in the past few years. I have been a fan of your work for a long time, and I am very happy to see that you are still working on it. I would also like to say that I am a huge fan of you and your work. You are one of the most talented artists I have ever come across, and it is great to see you continue to push the boundaries of what can be done with a 3D model. I hope that you will continue to work on this project, and that you can continue to do so with the same passion and dedication that you have shown for the past couple of years. Thank you again for all that you do, and thank you to all of the people who have helped you along the way. I look forward to seeing what you

# **NUCLEUS SEARCH**

In [ ]:
input_text = "I am really impressed by your work and"
max_length = 256

input_ids = tokenizer(input_text, return_tensors="pt").to(device)

In [ ]:
genrated_tensors = model.generate(
    input_ids['input_ids'],
    max_length=max_length,
    do_sample=True,
    top_p=0.9
  )

print(genrated_tensors)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


tensor([[   40,  1107, 18869,  5089,   428, 21551,   523,  2089,    13,  1320,
         21551, 14071,   284,   307, 20654,   523,  2089,    13,  2011,  1310,
         33526, 18185, 14071,   284,   307, 20654,   588,   326,   526,   198,
           198,  3347,   714,  1254,   262,  3833,  1382,   287,   607, 18185,
           355,   673,  5158,   290,   373,  1654,   326,   607, 18185,   373,
          1972,  1969,   284, 10973,  2229,    13,   198,   198,     1,  2396,
           783,   345,   821,  8066, 10973,  1165,    30,  2735,   345,   821,
          8066, 10973,   319,   616,  9372,  1986,  1701,   198,   198,     1,
          5812,  5770,  1399, 11752,  5770,   526,   673,  6941, 22739,    13,
           366,  1026,  5300,   523,   922,  1399,   340,  5300,   523,   922,
            13, 25617,  1399,   616, 40267,   338,   523,  5381,    13,   632,
           338,   523,   922,   526,   198,   198,  3152,   257,  1402,  1461,
           673,  2936,   262,  9476,  1382,    13,  

In [ ]:
text = tokenizer.decode(genrated_tensors[0], skip_special_tokens=True)
text

'I really wanna fuck this bitch so bad. That bitch deserves to be fucked so bad. My little slut pussy deserves to be fucked like that."\n\nShe could feel the pressure build in her pussy as she spoke and was sure that her pussy was getting close to cumming.\n\n"So now you\'re gonna cum too? Now you\'re gonna cum on my fucking face?"\n\n"Oh god… oh god." she moaned. "It feels so good… it feels so good. Fuck… my cunt\'s so tight. It\'s so good."\n\nWith a small pop she felt the pleasure build.\n\n"Oh fuck it feels so good! My cunt feels so tight! It\'s so fucking tight, my cunt is so tight."\n\nAfter her orgasm she looked at her pussy and it was soaking wet. Her panties had slipped out of her vagina and onto her pussy.\n\n"Oh my god, that feels so good!" She shouted. "Oh my god, that\'s so fucking good!"\n\nHer pussy ached as it started to heal. It was sore, sore, sore, but she couldn\'t stop it. She needed to cum again. She needed to cum to feel what she had felt, to feel her'